In [ ]:
from random import shuffle
from pandas import read_csv
from sklearn import metrics, preprocessing
import numpy as np

In [ ]:
def score1(a,b):
    counter = 0
    for t in range(len(a)):
        if list(a[t])==list(b[t]):
            counter += 1
            
    return counter


def split_data(data, end_of_test_data):
    return(data[:,end_of_test_data:], data[:,:end_of_test_data])


def standarization(data):
    return preprocessing.scale(data)

In [ ]:
#get train data
data = read_csv("train.csv", delimiter=',').as_matrix()
shuffle(data)
x,y = split_data(data,14)
x = standarization(x)

In [ ]:
#classifier
import torch
from torch import nn, autograd, optim
from torch.nn import functional as F

In [ ]:
class BiClassMultiLabel(nn.Module):
    def __init__(self,D_in, D_H1, D_H2,D_H3, D_out):
        super(BiClassMultiLabel,self).__init__()
        self.layer1 =  nn.Linear(bias=True, in_features=D_in, out_features=D_H1)
        nn.init.xavier_normal(self.layer1.weight)
        
        self.layer2 =  nn.Linear(bias=True, in_features=D_H1, out_features=D_H2)
        nn.init.xavier_normal( self.layer2.weight)
        
        self.layer3 =  nn.Linear(bias=True, in_features=D_H2, out_features=D_H3)
        nn.init.xavier_normal( self.layer3.weight)
        
        self.layer4 =  nn.Linear(bias = True, in_features=D_H3, out_features=D_out)
        nn.init.xavier_normal(self.layer4.weight)
        
        self.bn1 = nn.BatchNorm1d(D_H1)
        self.bn2 = nn.BatchNorm1d(D_H2)
        self.bn3 = nn.BatchNorm1d(D_H3)
   



    def forward(self, vec_in):
        o = self.bn1(F.rrelu(self.layer1(vec_in)))
        o = F.dropout(input=o,p=0.3, training=True)
        
        o = self.bn2(F.selu(input=self.layer2(o)))
        o = F.dropout(input=o,p=0.4, training=True)
        
        o = self.bn3(F.rrelu(input=self.layer3(o)))
        o = F.dropout(input=o,p=0.3, training=True)
        
        o = F.sigmoid(self.layer4(o))
        
        return o
    
    
    
    
    
def train(x_tr, y_tr):
    data1 = torch.from_numpy(x_tr).type(torch.FloatTensor)
    data2 = torch.from_numpy(y_tr).type(torch.FloatTensor)
    x_tr = autograd.Variable(data1, requires_grad = True)
    y_tr = autograd.Variable(data2, requires_grad = False)

    for epoch in range(200):
        optimizer.zero_grad()
        y_pred = model(x_tr)

        loss = F.binary_cross_entropy(y_pred,y_tr)
        loss.backward()

        optimizer.step()
        print("########### EPOCH: ",epoch,"########### LOSS: ",loss)
    
    
    
    
    
def valid(x_te, y_te):
    x = torch.from_numpy(x_te).type(torch.FloatTensor)
    x = autograd.Variable(x, requires_grad = True)
    y_pred = model(x).data.numpy()
    y_pred = np.around(y_pred)

    s1 = score1(y_te, y_pred)
    print("score1: ", s1)
    a = metrics.accuracy_score(y_pred, y_te)
    print("acc: ", a)
    h = metrics.hamming_loss(y_pred, y_te)
    print("ham: ", h)
    jacc = metrics.jaccard_similarity_score(y_pred, y_te)
    print("jaccard: ", jacc)

    return jacc


In [ ]:
torch.manual_seed(11317)

model = BiClassMultiLabel(103,110,70, 55,14)
optimizer = optim.Adam(params=model.parameters() ,lr=1e-3)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, shuffle= True)
accuracy_mean = 0
for tr, te in kf.split(x,y):
    x_tr , x_te = x[tr], x[te]
    y_tr , y_te = y[tr], y[te]
    
    train(x_tr,y_tr)
    accuracy_mean += valid(x_te, y_te)
print(accuracy_mean/2)


In [ ]:
# train(x,y)

In [ ]:
#get test data & valid
xte = read_csv("test.csv", delimiter=',').as_matrix()
xte = standarization(xte)
yte = read_csv("wynik_poprawny.csv").as_matrix()
    
valid(xte, yte)
